# Import library

In [ ]:
import tensorflow as tf
from tfx.components import (
    CsvExampleGen, 
    StatisticsGen, 
    SchemaGen, 
    ExampleValidator, 
    Transform, 
    Trainer,
    Evaluator,
    Pusher,
    Tuner
)
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
from tensorflow import keras
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.proto import trainer_pb2, pusher_pb2

# Set Variable

Mendefinisikan variabel input

In [2]:
DATA_ROOT = "data"
MODULES_ROOT = "modules"
os.makedirs(MODULES_ROOT, exist_ok=True)
TRANSFORM_MODULE_FILE = os.path.join(MODULES_ROOT,"diabetes_pred_transform.py")
TRAINER_MODULE_FILE = os.path.join(MODULES_ROOT,"diabetes_pred_trainer.py")
TUNER_HPARAMS_FILE = os.path.join(MODULES_ROOT,"diabetes_pred_tuner.py")

Mendefinisikan variabel output

In [3]:
OUTPUT_BASE = "outputs"
PIPELINE_NAME = "umartils-pipeline"
SCHEMA_PIPELINE_NAME = "diabetes-tfdv-schema"
PIPELINE_ROOT = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join(OUTPUT_BASE, 'serving_model')


In [4]:
FEATURE = [
    'Disease',
    'Fever',
    'Cough',
    'Fatigue',
    'Difficulty_Breathing',
    'Age',
    'Gender',
    'Blood_Pressure',
    'Cholesterol_Level'
]
LABEL_KEY = 'Outcome_Variable'

In [5]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Example Gen

Membuat komponen Example Gen

In [6]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

Menjalankan komponen Example Gen

In [7]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

# Statistic Gen

Membuat dan menjalankan komponen Statistic Gen

In [8]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

Menampilkan visualisasi interaktif dari komponen Statistic Gen

In [9]:
interactive_context.show(statistics_gen.outputs["statistics"])

# Schema Gen

In [10]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)
 

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'Blood_Pressure',STRING,required,,'Blood_Pressure'
'Cholesterol_Level',STRING,required,,'Cholesterol_Level'
'Cough',STRING,required,,'Cough'
'Difficulty_Breathing',STRING,required,,'Difficulty_Breathing'
'Disease',BYTES,required,,-
'Fatigue',STRING,required,,'Fatigue'
'Fever',STRING,required,,'Fever'
'Gender',STRING,required,,'Gender'


,Values
Domain,
'Blood_Pressure',"'High', 'Low', 'Normal'"
'Cholesterol_Level',"'High', 'Low', 'Normal'"
'Cough',"'No', 'Yes'"
'Difficulty_Breathing',"'No', 'Yes'"
'Fatigue',"'No', 'Yes'"
'Fever',"'No', 'Yes'"
'Gender',"'Female', 'Male'"


# Example Validator

In [12]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [13]:
interactive_context.show(example_validator.outputs['anomalies'])

# Transform

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft

# Fitur kategorikal
CATEGORICAL_FEATURES = [
    'Disease',
    'Fever',
    'Cough',
    'Fatigue',
    'Difficulty_Breathing',
    'Gender',
    'Blood_Pressure',
    'Cholesterol_Level',
]

# Fitur numerik
NUMERICAL_FEATURES = [
    'Age'
]

# Label (tidak diproses di Transform)
LABEL_KEY = 'Outcome_Variable'

def preprocessing_fn(inputs):
    """TFX Transform preprocessing function"""
    outputs = {}

    print("=== TFX Transform preprocessing_fn ===")
    print(f"Input keys: {list(inputs.keys())}")
    
    # Transform fitur numerik - scale to 0-1
    for key in NUMERICAL_FEATURES:
        if key in inputs:
            print(f"Processing numerical feature: {key}")
            # Cast ke float32 dan scale ke 0-1
            numeric_feature = tf.cast(inputs[key], tf.float32)
            outputs[f'{key}'] = tft.scale_to_0_1(numeric_feature)
            print(f"✓ Created: {key}")
        else:
            print(f"⚠️  Numerical feature '{key}' not found in inputs")

    # Transform fitur kategorikal menjadi indeks integer
    for key in CATEGORICAL_FEATURES:
        if key in inputs:
            print(f"Processing categorical feature: {key}")
            
            # Handle different data types
            feature_values = inputs[key]
            
            # Convert to string if not already
            if feature_values.dtype == tf.string:
                # Jika sudah string, normalize ke lowercase untuk konsistensi
                string_feature = tf.strings.lower(feature_values)
            else:
                # Jika numerik atau boolean, convert ke string
                string_feature = tf.strings.as_string(feature_values)
            
            # Buat vocabulary dan apply ke feature
            # Ini akan membuat vocabulary file dan mengembalikan integer indices
            outputs[f'{key}'] = tft.compute_and_apply_vocabulary(
                string_feature,
                vocab_filename=f'{key}_vocab',  # Explicit vocabulary filename
                default_value=-1,  # Value for OOV (Out of Vocabulary)
                top_k=None,  # Include all unique values
                frequency_threshold=1,  # Minimum frequency to include in vocab
                num_oov_buckets=1  # Number of OOV buckets
            )
            print(f"✓ Created: {key} with vocabulary {key}_vocab")
        else:
            print(f"⚠️  Categorical feature '{key}' not found in inputs")

    # Label tidak ditransform → diteruskan saja
    if LABEL_KEY in inputs:
        outputs[LABEL_KEY] = tf.cast(inputs[LABEL_KEY], tf.int64)
        print(f"✓ Label '{LABEL_KEY}' passed through as int64")
    else:
        print(f"⚠️  Label '{LABEL_KEY}' not found in inputs")

    print(f"Transform output keys: {list(outputs.keys())}")
    print("=== End Transform preprocessing_fn ===")
    
    return outputs


Overwriting modules\diabetes_pred_transform.py


In [19]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

=== TFX Transform preprocessing_fn ===
Input keys: ['Age', 'Blood_Pressure', 'Cholesterol_Level', 'Cough', 'Difficulty_Breathing', 'Disease', 'Fatigue', 'Fever', 'Gender', 'Outcome_Variable']
Processing numerical feature: Age
✓ Created: Age
Processing categorical feature: Disease


✓ Created: Disease with vocabulary Disease_vocab
Processing categorical feature: Fever


✓ Created: Fever with vocabulary Fever_vocab
Processing categorical feature: Cough


✓ Created: Cough with vocabulary Cough_vocab
Processing categorical feature: Fatigue


✓ Created: Fatigue with vocabulary Fatigue_vocab
Processing categorical feature: Difficulty_Breathing


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

# Trainer

In [20]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow_transform.tf_metadata import schema_utils
from tfx_bsl.public import tfxio
import os

# Feature definitions - pastikan ini sama dengan transform module
NUMERICAL_FEATURES = ['Age']
CATEGORICAL_FEATURES = [
    'Disease',
    'Fever',
    'Cough',
    'Fatigue',
    'Difficulty_Breathing',
    'Gender',
    'Blood_Pressure',
    'Cholesterol_Level',
]
LABEL_KEY = 'Outcome_Variable'

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""
    model.tft_layer = tf_transform_output.transform_features_layer()
    # Tentukan input signature
    input_signature = [tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')]
    @tf.function(input_signature=input_signature)
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY, None)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)
    return serve_tf_examples_fn.get_concrete_function()

def _get_transform_features_signature(model, tf_transform_output):
    """Gets a signature for transforming features."""
    model.tft_layer = tf_transform_output.transform_features_layer()
    transformed_feature_spec = tf_transform_output.transformed_feature_spec()
    input_signature = [
        {
            key: tf.TensorSpec(
                shape=(None, 1) if spec.dtype == tf.float32 else (None, 1),
                dtype=spec.dtype,
                name=key
            )
            for key, spec in transformed_feature_spec.items()
            if key != LABEL_KEY
        }
    ]
    @tf.function(input_signature=input_signature)
    def transform_features_fn(features):
        transformed_features = model.tft_layer(features)
        return model(transformed_features)
    return transform_features_fn.get_concrete_function()


def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size=32):
    """Loads TFRecord dataset for training/eval."""
    try:
        transformed_feature_spec = tf_transform_output.transformed_feature_spec()
        
        dataset = data_accessor.tf_dataset_factory(
            file_pattern,
            tfxio.TensorFlowDatasetOptions(
                batch_size=batch_size,
                label_key=LABEL_KEY,
                shuffle=True,
                shuffle_buffer_size=1000
            ),
            schema=tf_transform_output.transformed_metadata.schema
        )
        
        return dataset.repeat()
        
    except Exception as e:
        print(f"Error in _input_fn: {e}")
        raise

def _build_keras_model(tf_transform_output):
    """Build and compile Keras model."""
    
    # Get transformed feature spec
    transformed_feature_spec = tf_transform_output.transformed_feature_spec()
    
    print(f"Available transformed features: {list(transformed_feature_spec.keys())}")
    
    # Input layers untuk semua fitur yang sudah di-transform
    inputs = {}
    
    # Handle numerical features - sesuaikan dengan transform output
    numerical_inputs = []
    for feature in NUMERICAL_FEATURES:
        # Nama yang seharusnya ada berdasarkan transform module
        transformed_name = f"{feature}"
        
        if transformed_name in transformed_feature_spec:
            inputs[transformed_name] = keras.layers.Input(
                shape=(1,), name=transformed_name, dtype=tf.float32
            )
            numerical_inputs.append(inputs[transformed_name])
            print(f"✓ Found numerical feature: {transformed_name}")
        else:
            print(f"⚠️  Expected numerical feature '{transformed_name}' not found")
            print(f"Available features: {list(transformed_feature_spec.keys())}")
    
    # Handle categorical features - sesuaikan dengan transform output
    embedded = []
    for feature in CATEGORICAL_FEATURES:
        # Nama yang seharusnya ada berdasarkan transform module
        transformed_name = f"{feature}"
        
        if transformed_name in transformed_feature_spec:
            # Coba dapatkan vocab size dengan berbagai cara
            vocab_size = None
            vocab_name = f"{feature}_vocab"  # Sesuai dengan transform module
            
            try:
                vocab_size = tf_transform_output.vocabulary_size_by_name(vocab_name)
                print(f"✓ Found vocab for '{feature}': actual size={vocab_size}")
            except ValueError as e:
                print(f"⚠️  Cannot get vocab size for '{vocab_name}': {e}")
                
                # Fallback: inspect the transformed feature spec untuk estimasi
                try:
                    feature_spec = transformed_feature_spec[transformed_name]
                    print(f"Feature spec for {transformed_name}: {feature_spec}")
                    
                    # Jika ada info shape, gunakan untuk estimasi
                    vocab_size = 50  # Conservative fallback
                    print(f"Using conservative fallback vocab size: {vocab_size}")
                except:
                    vocab_size = 50  # Safe fallback
            
            # PENTING: Tambahkan buffer yang cukup untuk OOV dan unexpected values
            # TFT biasanya menyimpan: 0=padding, vocab_values, then OOV buckets
            embedding_vocab_size = vocab_size + 10  # Tambah buffer yang lebih besar
            
            print(f"Setting embedding vocab size for '{feature}': {embedding_vocab_size}")
            
            # Buat input layer dan embedding
            inputs[transformed_name] = keras.layers.Input(
                shape=(1,), name=transformed_name, dtype=tf.int64
            )
            
            embed_dim = min(50, max(4, vocab_size // 2))
            x = keras.layers.Embedding(
                input_dim=embedding_vocab_size,  # Dengan buffer yang aman
                output_dim=embed_dim,
                mask_zero=False  # Tidak menggunakan mask untuk menghindari kompleksitas
            )(inputs[transformed_name])
            x = keras.layers.Reshape((embed_dim,))(x)
            embedded.append(x)
            
        else:
            print(f"⚠️  Expected categorical feature '{transformed_name}' not found")
            print(f"Available features: {list(transformed_feature_spec.keys())}")
    
    # Concatenate all inputs
    all_inputs = numerical_inputs + embedded
    
    if len(all_inputs) == 0:
        raise ValueError(f"No valid features found for model input. Available features: {list(transformed_feature_spec.keys())}")
    elif len(all_inputs) == 1:
        concatenated = all_inputs[0]
    else:
        concatenated = keras.layers.concatenate(all_inputs)
    
    print(f"✓ Model input created with {len(all_inputs)} feature groups")
    
    # Deep neural network layers
    x = keras.layers.Dense(128, kernel_regularizer=keras.regularizers.l2(1e-5))(concatenated)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha=0.1)(x)
    x = keras.layers.Dropout(0.2)(x)
    
    x = keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(1e-5))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha=0.1)(x)
    x = keras.layers.Dropout(0.2)(x)
    
    x = keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(1e-5))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha=0.1)(x)
    x = keras.layers.Dropout(0.15)(x)
    
    x = keras.layers.Dense(16, activation='relu')(x)
    
    # Output layer untuk binary classification
    outputs = keras.layers.Dense(1, activation='sigmoid', name='predictions')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=[
            keras.metrics.BinaryAccuracy(name='accuracy'),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )
    
    return model

def run_fn(fn_args: FnArgs):
    """Main training function that TFX will call."""
    
    print("=" * 50)
    print("Starting TFX Trainer...")
    print(f"Transform output: {fn_args.transform_output}")
    print(f"Train files: {fn_args.train_files}")
    print(f"Eval files: {fn_args.eval_files}")
    print(f"Serving model dir: {fn_args.serving_model_dir}")
    print("=" * 50)
    
    try:
        # Load TF Transform output
        tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
        print("✓ TF Transform output loaded successfully")
        
        # Create datasets
        train_dataset = _input_fn(
            fn_args.train_files, 
            fn_args.data_accessor, 
            tf_transform_output, 
            batch_size=32
        )
        print("✓ Training dataset created")
        
        eval_dataset = _input_fn(
            fn_args.eval_files, 
            fn_args.data_accessor, 
            tf_transform_output, 
            batch_size=32
        )
        print("✓ Evaluation dataset created")
        
        # Build model
        model = _build_keras_model(tf_transform_output)
        print("✓ Model built successfully")
        print("\nModel Summary:")
        model.summary()
        
        # Training configuration
        train_steps = fn_args.train_steps or 1000
        eval_steps = fn_args.eval_steps or 200
        
        # Calculate epochs and steps
        steps_per_epoch = min(100, max(10, train_steps // 10))
        epochs = max(1, train_steps // steps_per_epoch)
        validation_steps = min(50, max(5, eval_steps // 2))
        
        print(f"\nTraining Configuration:")
        print(f"Total train steps: {train_steps}")
        print(f"Steps per epoch: {steps_per_epoch}")
        print(f"Epochs: {epochs}")
        print(f"Validation steps: {validation_steps}")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss', 
                patience=5, 
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', 
                factor=0.5, 
                patience=3,
                verbose=1,
                min_lr=1e-7
            )
        ]
        
        # Train the model
        print("\nStarting training...")
        history = model.fit(
            train_dataset,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_data=eval_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks,
            verbose=1
        )
        print("✓ Training completed successfully")
        
        # Create serving signatures
        signatures = {
            'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
            'transform_features': _get_transform_features_signature(model, tf_transform_output)
        }
        
        # Save the model
        print(f"\nSaving model to: {fn_args.serving_model_dir}")
        model.save(
            fn_args.serving_model_dir, 
            save_format='tf', 
            signatures=signatures
        )
        print("✓ Model saved successfully!")
        
        # Print training history summary
        if history and history.history:
            final_loss = history.history.get('loss', [])[-1] if history.history.get('loss') else 'N/A'
            final_accuracy = history.history.get('accuracy', [])[-1] if history.history.get('accuracy') else 'N/A'
            final_val_loss = history.history.get('val_loss', [])[-1] if history.history.get('val_loss') else 'N/A'
            final_val_accuracy = history.history.get('val_accuracy', [])[-1] if history.history.get('val_accuracy') else 'N/A'
            
            print(f"\nFinal Training Metrics:")
            print(f"Loss: {final_loss}")
            print(f"Accuracy: {final_accuracy}")
            print(f"Val Loss: {final_val_loss}")
            print(f"Val Accuracy: {final_val_accuracy}")
        
        print("=" * 50)
        print("TFX Trainer completed successfully!")
        print("=" * 50)
        
    except Exception as e:
        print(f"❌ Error in training: {str(e)}")
        import traceback
        traceback.print_exc()
        raise

Overwriting modules\diabetes_pred_trainer.py


In [21]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=200)
)

print("Running Trainer with simplified model...")
interactive_context.run(trainer)
print("✓ Trainer completed successfully!")

Running Trainer with simplified model...


Starting TFX Trainer...
Transform output: outputs\umartils-pipeline\Transform\transform_graph\7
Train files: ['outputs\\umartils-pipeline\\Transform\\transformed_examples\\7\\Split-train\\*']
Eval files: ['outputs\\umartils-pipeline\\Transform\\transformed_examples\\7\\Split-eval\\*']
Serving model dir: outputs\umartils-pipeline\Trainer\model\8\Format-Serving
✓ TF Transform output loaded successfully
✓ Training dataset created
✓ Evaluation dataset created
Available transformed features: ['Age', 'Blood_Pressure', 'Cholesterol_Level', 'Cough', 'Difficulty_Breathing', 'Disease', 'Fatigue', 'Fever', 'Gender', 'Outcome_Variable']
✓ Found numerical feature: Age
✓ Found vocab for 'Disease': actual size=102
Setting embedding vocab size for 'Disease': 112
✓ Found vocab for 'Fever': actual size=2
Setting embedding vocab size for 'Fever': 12
✓ Found vocab for 'Cough': actual size=2
Setting embedding vocab size for 'Cough': 12
✓ Found vocab for 'Fatigue': actual size=2
Setting embedding vocab size

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.



Saving model to: outputs\umartils-pipeline\Trainer\model\8\Format-Serving
INFO:tensorflow:Assets written to: outputs\umartils-pipeline\Trainer\model\8\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\umartils-pipeline\Trainer\model\8\Format-Serving\assets


✓ Model saved successfully!

Final Training Metrics:
Loss: 0.036697983741760254
Accuracy: 0.9871875047683716
Val Loss: 1.2298498153686523
Val Accuracy: 0.7056249976158142
TFX Trainer completed successfully!
✓ Trainer completed successfully!


# Resolver

In [22]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

# Evaluator

In [ ]:
import tensorflow_model_analysis as tfma

def create_eval_config():
    return tfma.EvalConfig(
        model_specs=[
            tfma.ModelSpec(
                label_key=LABEL_KEY,        # Sesuai transform.py
                prediction_key='predictions',  # Sesuai nama output model, seringnya 'output_0' di Keras
            )
        ],
        slicing_specs=[
            tfma.SlicingSpec(),  # Overall metrics
            tfma.SlicingSpec(feature_keys=[
                'Gender',
                'Fever',
                'Blood_Pressure',
                ]),
        ],
        metrics_specs=[
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value':0.5}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value':0.0001})
                        )
                )
            ])
        ]
    )

In [24]:
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=create_eval_config()
)

interactive_context.run(evaluator)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 10
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [23]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

# Tuner

In [25]:
%%writefile {TUNER_HPARAMS_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.components.tuner.component import TunerFnResult
from tensorflow_transform.tf_metadata import schema_utils
from tfx_bsl.public import tfxio
import kerastuner as kt
from typing import Dict, Any, List
import os

# Feature definitions - sama dengan trainer module
NUMERICAL_FEATURES = ['Age']
CATEGORICAL_FEATURES = [
    'Disease',
    'Fever',
    'Cough',
    'Fatigue',
    'Difficulty_Breathing',
    'Gender',
    'Blood_Pressure',
    'Cholesterol_Level',
]
LABEL_KEY = 'Outcome_Variable'

def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size=32):
    """Loads TFRecord dataset for training/eval - sama dengan trainer."""
    try:
        transformed_feature_spec = tf_transform_output.transformed_feature_spec()
        
        dataset = data_accessor.tf_dataset_factory(
            file_pattern,
            tfxio.TensorFlowDatasetOptions(
                batch_size=batch_size,
                label_key=LABEL_KEY,
                shuffle=True,
                shuffle_buffer_size=1000
            ),
            schema=tf_transform_output.transformed_metadata.schema
        )
        
        return dataset.repeat()
        
    except Exception as e:
        print(f"Error in _input_fn: {e}")
        raise

def _build_keras_model_with_hyperparams(tf_transform_output, hparams):
    """Build Keras model with tunable hyperparameters."""
    
    # Get transformed feature spec
    transformed_feature_spec = tf_transform_output.transformed_feature_spec()
    print(f"Available transformed features: {list(transformed_feature_spec.keys())}")
    
    # Input layers untuk semua fitur yang sudah di-transform
    inputs = {}
    
    # Handle numerical features
    numerical_inputs = []
    for feature in NUMERICAL_FEATURES:
        transformed_name = f"{feature}"
        
        if transformed_name in transformed_feature_spec:
            inputs[transformed_name] = keras.layers.Input(
                shape=(1,), name=transformed_name, dtype=tf.float32
            )
            numerical_inputs.append(inputs[transformed_name])
            print(f"✓ Found numerical feature: {transformed_name}")
    
    # Handle categorical features dengan tunable embedding dimensions
    embedded = []
    for feature in CATEGORICAL_FEATURES:
        transformed_name = f"{feature}"
        
        if transformed_name in transformed_feature_spec:
            # Get vocabulary size
            vocab_size = None
            vocab_name = f"{feature}_vocab"
            
            try:
                vocab_size = tf_transform_output.vocabulary_size_by_name(vocab_name)
                print(f"✓ Found vocab for '{feature}': actual size={vocab_size}")
            except ValueError as e:
                print(f"⚠️  Cannot get vocab size for '{vocab_name}': {e}")
                vocab_size = 50  # Conservative fallback
            
            # Buffer untuk OOV values
            embedding_vocab_size = vocab_size + 10
            
            # Tunable embedding dimension
            embed_dim = hparams.Int(
                f'embed_dim_{feature}',
                min_value=4,
                max_value=min(64, max(8, vocab_size // 2)),
                default=min(32, max(4, vocab_size // 4))
            )
            
            print(f"Setting embedding for '{feature}': vocab_size={embedding_vocab_size}, embed_dim={embed_dim}")
            
            # Create input layer dan embedding
            inputs[transformed_name] = keras.layers.Input(
                shape=(1,), name=transformed_name, dtype=tf.int64
            )
            
            x = keras.layers.Embedding(
                input_dim=embedding_vocab_size,
                output_dim=embed_dim,
                mask_zero=False
            )(inputs[transformed_name])
            x = keras.layers.Reshape((embed_dim,))(x)
            embedded.append(x)
    
    # Concatenate all inputs
    all_inputs = numerical_inputs + embedded
    
    if len(all_inputs) == 0:
        raise ValueError(f"No valid features found for model input. Available: {list(transformed_feature_spec.keys())}")
    elif len(all_inputs) == 1:
        concatenated = all_inputs[0]
    else:
        concatenated = keras.layers.concatenate(all_inputs)
    
    print(f"✓ Model input created with {len(all_inputs)} feature groups")
    
    # Tunable architecture parameters
    num_layers = hparams.Int('num_layers', min_value=2, max_value=5, default=3)
    use_batch_norm = hparams.Boolean('use_batch_norm', default=True)
    use_dropout = hparams.Boolean('use_dropout', default=True)
    activation_type = hparams.Choice('activation', ['relu', 'leaky_relu', 'elu'], default='leaky_relu')
    
    # Build tunable deep neural network
    x = concatenated
    
    for i in range(num_layers):
        # Tunable layer sizes - decreasing pattern
        if i == 0:  # First layer
            units = hparams.Int('units_first', min_value=64, max_value=256, step=32, default=128)
        elif i == num_layers - 1:  # Last layer
            units = hparams.Int('units_last', min_value=16, max_value=64, step=8, default=32)
        else:  # Middle layers
            units = hparams.Int(f'units_middle_{i}', min_value=32, max_value=128, step=16, default=64)
        
        # Dense layer dengan tunable L2 regularization
        l2_reg = hparams.Float('l2_regularization', min_value=1e-6, max_value=1e-3, sampling='log', default=1e-5)
        x = keras.layers.Dense(units, kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
        
        # Optional batch normalization
        if use_batch_norm:
            x = keras.layers.BatchNormalization()(x)
        
        # Tunable activation
        if activation_type == 'leaky_relu':
            alpha = hparams.Float('leaky_relu_alpha', min_value=0.01, max_value=0.3, default=0.1)
            x = keras.layers.LeakyReLU(alpha=alpha)(x)
        elif activation_type == 'elu':
            x = keras.layers.ELU()(x)
        else:  # relu
            x = keras.layers.ReLU()(x)
        
        # Optional dropout dengan tunable rate
        if use_dropout and i < num_layers - 1:  # No dropout on last layer
            dropout_rate = hparams.Float(f'dropout_rate_{i}', min_value=0.1, max_value=0.5, default=0.2)
            x = keras.layers.Dropout(dropout_rate)(x)
    
    # Final dense layer sebelum output
    final_units = hparams.Int('final_units', min_value=8, max_value=32, step=4, default=16)
    x = keras.layers.Dense(final_units, activation='relu')(x)
    
    # Output layer untuk binary classification
    outputs = keras.layers.Dense(1, activation='sigmoid', name='predictions')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Tunable optimizer dan learning rate
    learning_rate = hparams.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log', default=1e-3)
    optimizer_type = hparams.Choice('optimizer', ['adam', 'adamw', 'rmsprop'], default='adam')
    
    if optimizer_type == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_type == 'adamw':
        weight_decay = hparams.Float('weight_decay', min_value=1e-5, max_value=1e-2, sampling='log', default=1e-4)
        optimizer = keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)
    else:  # rmsprop
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    # Compile model
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            keras.metrics.BinaryAccuracy(name='accuracy'),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )
    
    print(f"✓ Model built with tunable hyperparameters:")
    print(f"  - Layers: {num_layers}")
    print(f"  - Optimizer: {optimizer_type}")
    print(f"  - Learning rate: {learning_rate}")
    print(f"  - Batch norm: {use_batch_norm}")
    print(f"  - Dropout: {use_dropout}")
    
    return model
 
def tuner_fn(fn_args) -> TunerFnResult:
    """TFX Tuner function untuk hyperparameter optimization."""
    
    print("=" * 60)
    print("Starting TFX Tuner for Medical Diagnosis Model...")
    print(f"Transform output: {fn_args.transform_graph_path}")
    print(f"Train files: {fn_args.train_files}")
    print(f"Eval files: {fn_args.eval_files}")
    print(f"Working directory: {fn_args.working_dir}")
    print("=" * 60)
    
    try:
        # Load TF Transform output
        tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
        print("✓ TF Transform output loaded successfully")
        
        # Create datasets untuk tuning
        train_dataset = _input_fn(
            fn_args.train_files[0],
            fn_args.data_accessor,
            tf_transform_output,
            batch_size=32
        )
        print("✓ Training dataset created")
        
        eval_dataset = _input_fn(
            fn_args.eval_files[0],
            fn_args.data_accessor,
            tf_transform_output,
            batch_size=32
        )
        print("✓ Evaluation dataset created")
        
        # Hypermodel function
        def build_model(hp):
            return _build_keras_model_with_hyperparams(tf_transform_output, hp)
        
        # Setup tuner - menggunakan Bayesian Optimization untuk efisiensi
        tuner = kt.BayesianOptimization(
            hypermodel=build_model,
            objective=kt.Objective('val_auc', direction='max'),  # Optimize AUC untuk medical diagnosis
            max_trials=15,  # Reasonable number untuk medical model
            num_initial_points=3,
            directory=fn_args.working_dir,
            project_name='medical_diagnosis_tuning',
            seed=42,
            overwrite=True
        )
        
        print("✓ Bayesian Optimization tuner initialized")
        print(f"  - Max trials: 15")
        print(f"  - Objective: Maximize validation AUC")
        print(f"  - Initial random points: 3")
        
        # Calculate training parameters
        train_steps = fn_args.train_steps or 1000
        eval_steps = fn_args.eval_steps or 200
        
        # For tuning, use shorter training to speed up search
        tuning_epochs = 15  # Shorter untuk tuning
        steps_per_epoch = min(50, max(10, train_steps // 20))  # Fewer steps per epoch
        validation_steps = min(25, max(5, eval_steps // 4))
        
        print(f"\nTuning Configuration:")
        print(f"Training epochs per trial: {tuning_epochs}")
        print(f"Steps per epoch: {steps_per_epoch}")
        print(f"Validation steps: {validation_steps}")
        
        # Enhanced callbacks untuk tuning
        def get_callbacks():
            return [
                # Early stopping untuk avoid overfitting during tuning
                keras.callbacks.EarlyStopping(
                    monitor='val_auc',
                    patience=5,
                    restore_best_weights=True,
                    verbose=0,
                    mode='max'
                ),
                
                # Reduce learning rate untuk fine-tuning
                keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.7,
                    patience=3,
                    verbose=0,
                    min_lr=1e-7
                ),
                
                # Stop unpromising trials early
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=7,
                    restore_best_weights=True,
                    verbose=0,
                    mode='min'
                )
            ]
        
        # Return TunerFnResult
        
        return TunerFnResult(
            tuner=tuner,
            fit_kwargs={
                'x': train_dataset,
                'validation_data': eval_dataset,
                'steps_per_epoch': steps_per_epoch,
                'validation_steps': validation_steps,
                'epochs': tuning_epochs,
                'callbacks': get_callbacks(),
                'verbose': 1
            }
        )
        
    except Exception as e:
        print(f"❌ Error in tuner setup: {str(e)}")
        import traceback
        traceback.print_exc()
        raise

# # Advanced tuner function dengan lebih banyak strategi
# def advanced_tuner_fn(fn_args: FnArgs):
#     """Advanced tuner dengan multiple objectives dan strategi."""
    
#     print("=" * 60)
#     print("Starting ADVANCED TFX Tuner for Medical Diagnosis...")
#     print("=" * 60)
    
#     try:
#         tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
        
#         # Datasets
#         train_dataset = _input_fn(
#             fn_args.train_files, 
#             fn_args.data_accessor, 
#             tf_transform_output, 
#             batch_size=32
#         )
#         print("✓ Training dataset created")
        
#         eval_dataset = _input_fn(
#             fn_args.eval_files, 
#             fn_args.data_accessor, 
#             tf_transform_output, 
#             batch_size=32
#         )
#         print("✓ Evaluation dataset created")
        
#         # Advanced hypermodel dengan conditional parameters
#         def advanced_build_model(hp):
#             model = _build_keras_model_with_hyperparams(tf_transform_output, hp)
            
#             # Advanced tuning: conditional architectures
#             use_residual = hp.Boolean('use_residual_connections', default=False)
            
#             if use_residual:
#                 print("🔧 Using residual connections in model")
#                 # Implementasi residual connections bisa ditambahkan di sini
            
#             # Class weight balancing untuk medical data
#             use_class_weights = hp.Boolean('use_class_weights', default=True)
#             if use_class_weights:
#                 # Untuk medical diagnosis, biasanya imbalanced
#                 class_weight = {
#                     0: hp.Float('class_weight_negative', 0.3, 1.0, default=0.5),
#                     1: hp.Float('class_weight_positive', 1.0, 3.0, default=1.5)
#                 }
#                 print(f"🔧 Using class weights: {class_weight}")
            
#             return model
        
#         # Multi-objective tuner
#         tuner = kt.BayesianOptimization(
#             hypermodel=advanced_build_model,
#             objective=[
#                 kt.Objective('val_auc', direction='max'),           # Primary: AUC
#                 kt.Objective('val_precision', direction='max'),     # Secondary: Precision
#                 kt.Objective('val_recall', direction='max')         # Secondary: Recall
#             ],
#             max_trials=20,
#             num_initial_points=4,
#             directory=fn_args.working_dir,
#             project_name='advanced_medical_tuning',
#             seed=42,
#             overwrite=True
#         )
        
#         print("✓ Advanced multi-objective tuner initialized")
        
#         # Enhanced callbacks dengan medical-specific monitoring
#         def get_advanced_callbacks():
#             return [
#                 # Medical-specific early stopping
#                 keras.callbacks.EarlyStopping(
#                     monitor='val_auc',
#                     patience=6,
#                     restore_best_weights=True,
#                     verbose=1,
#                     mode='max',
#                     min_delta=0.001  # Minimal improvement
#                 ),
                
#                 # Precision-Recall monitoring
#                 keras.callbacks.EarlyStopping(
#                     monitor='val_precision',
#                     patience=8,
#                     restore_best_weights=True,
#                     verbose=0,
#                     mode='max',
#                     min_delta=0.005
#                 ),
                
#                 # Learning rate scheduling
#                 keras.callbacks.ReduceLROnPlateau(
#                     monitor='val_loss',
#                     factor=0.8,
#                     patience=4,
#                     verbose=1,
#                     min_lr=1e-7,
#                     cooldown=2
#                 ),
                
#                 # Custom callback untuk medical metrics
#                 MedicalMetricsCallback()
#             ]
        
#         from tfx.components.tuner.component import TunerFnResult
        
#         return TunerFnResult(
#             tuner=tuner,
#             fit_kwargs={
#                 'x': train_dataset,
#                 'validation_data': eval_dataset,
#                 'steps_per_epoch': 40,  # Efficient tuning
#                 'validation_steps': 20,
#                 'epochs': 20,
#                 'callbacks': get_advanced_callbacks(),
#                 'verbose': 1
#             }
#         )
        
#     except Exception as e:
#         print(f"❌ Error in advanced tuner: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         raise

# Custom callback untuk medical-specific metrics
class MedicalMetricsCallback(keras.callbacks.Callback):
    """Custom callback untuk monitoring medical diagnosis metrics."""
    
    def __init__(self):
        super().__init__()
        self.best_f1 = 0.0
        self.best_balance = 0.0
    
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            return
        
        # Calculate F1 score
        precision = logs.get('val_precision', 0)
        recall = logs.get('val_recall', 0)
        
        if precision + recall > 0:
            f1_score = 2 * (precision * recall) / (precision + recall)
            logs['val_f1_score'] = f1_score
            
            if f1_score > self.best_f1:
                self.best_f1 = f1_score
                print(f"\n🎯 New best F1 score: {f1_score:.4f}")
        
        # Calculate balanced accuracy approximation
        accuracy = logs.get('val_accuracy', 0)
        auc = logs.get('val_auc', 0)
        balanced_score = (accuracy + auc) / 2
        
        if balanced_score > self.best_balance:
            self.best_balance = balanced_score
            print(f"⚖️  New best balanced score: {balanced_score:.4f}")

# Hyperparameter analysis utilities
def analyze_tuning_results(tuner_output_path, top_n=5):
    """Analyze dan extract best hyperparameters dari tuning results."""
    
    try:
        # Load tuner results
        tuner = kt.BayesianOptimization(
            hypermodel=lambda hp: None,  # Dummy hypermodel
            objective='val_auc',
            directory=tuner_output_path,
            project_name='medical_diagnosis_tuning'
        )
        
        # Get best trials
        best_trials = tuner.oracle.get_best_trials(num_trials=top_n)
        
        print(f"\n{'='*60}")
        print(f"TOP {top_n} HYPERPARAMETER COMBINATIONS")
        print(f"{'='*60}")
        
        results = []
        for i, trial in enumerate(best_trials):
            print(f"\n🏆 TRIAL {i+1} (Score: {trial.score:.4f})")
            print("-" * 40)
            
            # Extract hyperparameters
            hparams = trial.hyperparameters
            trial_result = {
                'trial_id': trial.trial_id,
                'score': trial.score,
                'hyperparameters': {}
            }
            
            # Kategorisasi hyperparameters
            architecture_params = {}
            training_params = {}
            regularization_params = {}
            
            for param_name in hparams.space:
                param_value = hparams.get(param_name)
                trial_result['hyperparameters'][param_name] = param_value
                
                if any(keyword in param_name.lower() for keyword in ['units', 'layers', 'embed', 'final']):
                    architecture_params[param_name] = param_value
                elif any(keyword in param_name.lower() for keyword in ['learning', 'optimizer', 'weight_decay']):
                    training_params[param_name] = param_value
                elif any(keyword in param_name.lower() for keyword in ['dropout', 'l2', 'batch_norm']):
                    regularization_params[param_name] = param_value
            
            # Print categorized parameters
            if architecture_params:
                print("🏗️  Architecture:")
                for param, value in architecture_params.items():
                    print(f"   {param}: {value}")
            
            if training_params:
                print("🎯 Training:")
                for param, value in training_params.items():
                    print(f"   {param}: {value}")
                    
            if regularization_params:
                print("🛡️  Regularization:")
                for param, value in regularization_params.items():
                    print(f"   {param}: {value}")
            
            results.append(trial_result)
        
        print(f"\n{'='*60}")
        print("HYPERPARAMETER INSIGHTS")
        print(f"{'='*60}")
        
        # Analyze patterns
        if results:
            # Best learning rates
            learning_rates = [r['hyperparameters'].get('learning_rate', 0) for r in results if 'learning_rate' in r['hyperparameters']]
            if learning_rates:
                print(f"📊 Learning Rate Range: {min(learning_rates):.2e} - {max(learning_rates):.2e}")
                print(f"📊 Median Learning Rate: {sorted(learning_rates)[len(learning_rates)//2]:.2e}")
            
            # Best architectures
            num_layers = [r['hyperparameters'].get('num_layers', 0) for r in results if 'num_layers' in r['hyperparameters']]
            if num_layers:
                from collections import Counter
                layer_counts = Counter(num_layers)
                print(f"🏗️  Most common layer count: {layer_counts.most_common(1)[0][0]} layers")
        
        return results
        
    except Exception as e:
        print(f"❌ Error analyzing results: {e}")
        return []

# Export best hyperparameters untuk production use
def export_best_hyperparameters(tuning_results, output_file='best_hyperparams.json'):
    """Export best hyperparameters ke JSON file untuk production."""
    
    import json
    
    if not tuning_results:
        print("❌ No tuning results to export")
        return
    
    best_config = tuning_results[0]  # Best trial
    
    export_data = {
        'model_version': '1.0',
        'tuning_date': str(tf.timestamp()),
        'best_score': float(best_config['score']),
        'hyperparameters': best_config['hyperparameters'],
        'model_config': {
            'task_type': 'binary_classification',
            'domain': 'medical_diagnosis',
            'features': {
                'numerical': NUMERICAL_FEATURES,
                'categorical': CATEGORICAL_FEATURES
            }
        }
    }
    
    try:
        with open(output_file, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        print(f"✅ Best hyperparameters exported to: {output_file}")
        print(f"🎯 Best validation AUC: {best_config['score']:.4f}")
        
        return output_file
        
    except Exception as e:
        print(f"❌ Error exporting hyperparameters: {e}")
        return None

Overwriting modules\diabetes_pred_tuner.py


In [27]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_HPARAMS_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    
    train_args=trainer_pb2.TrainArgs(num_steps=500),
    eval_args=trainer_pb2.EvalArgs(num_steps=200)
)
interactive_context.run(tuner)


ExecutionResult(
    component_id: Tuner
    execution_id: 12
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

Melatih ulang model menggunakan best_parameter hasil dari proses hyperparameter tuning menggunakan komponen Tuner

In [28]:
trainer_tuned = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=200),
    custom_config={'tuner': tuner.outputs['best_hyperparameters']}
)

print("Running Trainer with simplified model...")
interactive_context.run(trainer_tuned)
print("✓ Trainer completed successfully!")

Running Trainer with simplified model...


Starting TFX Trainer...
Transform output: outputs\umartils-pipeline\Transform\transform_graph\7
Train files: ['outputs\\umartils-pipeline\\Transform\\transformed_examples\\7\\Split-train\\*']
Eval files: ['outputs\\umartils-pipeline\\Transform\\transformed_examples\\7\\Split-eval\\*']
Serving model dir: outputs\umartils-pipeline\Trainer\model\13\Format-Serving
✓ TF Transform output loaded successfully
✓ Training dataset created
✓ Evaluation dataset created
Available transformed features: ['Age', 'Blood_Pressure', 'Cholesterol_Level', 'Cough', 'Difficulty_Breathing', 'Disease', 'Fatigue', 'Fever', 'Gender', 'Outcome_Variable']
✓ Found numerical feature: Age
✓ Found vocab for 'Disease': actual size=102
Setting embedding vocab size for 'Disease': 112
✓ Found vocab for 'Fever': actual size=2
Setting embedding vocab size for 'Fever': 12
✓ Found vocab for 'Cough': actual size=2
Setting embedding vocab size for 'Cough': 12
✓ Found vocab for 'Fatigue': actual size=2
Setting embedding vocab siz

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.



Saving model to: outputs\umartils-pipeline\Trainer\model\13\Format-Serving
INFO:tensorflow:Assets written to: outputs\umartils-pipeline\Trainer\model\13\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\umartils-pipeline\Trainer\model\13\Format-Serving\assets


✓ Model saved successfully!

Final Training Metrics:
Loss: 0.05288904532790184
Accuracy: 0.9840624928474426
Val Loss: 1.4381797313690186
Val Accuracy: 0.6206250190734863
TFX Trainer completed successfully!
✓ Trainer completed successfully!


# Pusher

In [29]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR
        )
    )
)
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 14
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))